In [ ]:
!pip install jupyter_dash

In [3]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd


df = pd.read_csv('/content/zomato.csv')
dfc=pd.read_csv('/content/Country-Code.csv')
app = JupyterDash(__name__)

In [4]:
df_merge = pd.merge(df, dfc, how='inner', on = 'Country Code')
#display(df_merge.head())
dfr=df_merge
#dfr["Rupees"]=dfr["Average Cost for two"]*75
#display(dfr.head())
display(dfr['Currency'].unique())

array(['Botswana Pula(P)', 'Brazilian Real(R$)', 'Dollar($)',
       'Emirati Diram(AED)', 'Indian Rupees(Rs.)',
       'Indonesian Rupiah(IDR)', 'NewZealand($)', 'Pounds(專)',
       'Qatari Rial(QR)', 'Rand(R)', 'Sri Lankan Rupee(LKR)',
       'Turkish Lira(TL)'], dtype=object)

In [5]:
#current conversion
def toinr(cur,cost):
  if cur=='Botswana Pula(P)':
    return cost*6.40
  elif cur=='Brazilian Real(R$)':
    return cost*15.66
  elif cur=='Dollar($)':
    return cost*81.56
  elif cur=='Emirati Diram(AED)':
    return cost*22.21
  elif cur=='Indian Rupees(Rs.)':
    return cost
  elif cur=='Indonesian Rupiah(IDR)':
    return cost*0.0055
  elif cur=='NewZealand($)':
    return cost*53.15
  elif cur=='Pounds(專)':
    return cost*101.13
  elif cur=='Qatari Rial(QR)':
    return cost*22.39
  elif cur=='Rand(R)':
    return cost*0.21
  elif cur=='Sri Lankan Rupee(LKR)':
    return cost*0.22
  elif cur=='Turkish Lira(TL)':
    return cost*0.23

In [ ]:
dfr.rename(columns = {'Average Cost for two':'AvgCost'}, inplace = True)
dfr['Amount_In_Rupees'] = dfr.apply( lambda x: toinr( x.Currency, x.AvgCost ), axis = 1)
#df['exchangerate'] = df['USD'] / df['Amount']
display(dfr)

In [7]:
from dash import Dash, dcc, html, Input, Output

In [8]:
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output
 
colors = {
   'background': '#4FE377',
   'fullBG':'D8D8D8',
   'zom':'#B22222',
   'graph':'A711F1',
   'text': '#111111',
   'textW':'FFFFFF'
}

#figures
fig1 = px.area(df_merge.groupby("Country")['Aggregate rating'].mean(),y="Aggregate rating",title= 'Aggregate Ratings vs Country')
fig2 = px.bar(dfr,x='Currency',y='Amount_In_Rupees',title= 'Rupee vs Other Currencies', text_auto=True)
fig3 = px.bar(dfr,x='Cuisines', y='Amount_In_Rupees',title= 'Rupee vs Cuisines', text_auto=True)
fig4=px.pie(dfr,names='Has Online delivery',title= 'Online Delivery vs Offline Delivery')
fig5=px.area(df_merge.groupby("Country")['Votes'].sum(),y="Votes",title= 'No of Ratings vs Country')
d1=dfr[dfr["Country"]=="India"]
d2=d1[d1['Has Online delivery']=='Yes']
d3=d1[d1['Has Online delivery']=='No']
fig6=px.bar(d2,x='City', y='Amount_In_Rupees',text_auto=True,title= 'Which part of India spends more on online deliver?')
fig7=px.bar(d3,x='City', y='Amount_In_Rupees',text_auto=True,title= 'Which part of India spends more on Dine-in?')
fig8 = px.area(d1.groupby("City")['Amount_In_Rupees'].mean(),y="Amount_In_Rupees",title= 'Cost of living')
#app = Dash(__name__)
app = JupyterDash(__name__)
app.layout = html.Div([
   html.H1(children="Zomato",
           style={
           'textAlign': 'center',
           'color': colors['textW'],
           'backgroundColor': colors['zom']
       }),
   
   dcc.Dropdown(
           options=[
               {'label': i, 'value': i}for i in dfc['Country']
           ],
           #value=['BNG', 'HYR'],
           placeholder="Select a country",
           id="my-input"
       ),
   html.Br(),
   html.Div(dcc.Dropdown(
                    id='my-output',
                    multi=False,
                    clearable=True,
                    disabled=False,
                    options=[],
                    placeholder="Select a city",
                ),
            ),
  html.Br(),
  html.Div([
      dcc.Graph(id='graph',
                figure=fig1,
                ),
      dcc.Graph(id='graph_rating',
                figure=fig5
                ),
      dcc.Graph(id='Pie_delivery',
                figure=fig4
                ),
  ],style={'display': 'flex', 'flex-direction': 'row'}),

  html.Div([
      dcc.Graph(id='Pie_cost',
                figure=fig6
                ),
      dcc.Graph(id='Pie_costofliving',
                figure=fig7
                ),
            
  ],style={'display': 'flex', 'flex-direction': 'row'}),
  dcc.Graph(id='livingexpense',
                figure=fig8
                ),
  
  html.Div([
      dcc.Graph(id='graph2',
                figure=fig2,
                style={'backgroundColor': 'C94EFA'}
                ),
  ]),
  html.Div([
      dcc.Graph(id='graph3',
                figure=fig3,
                style={'backgroundColor': 'C94EFA'}
                )
  ])
 
],style={'backgroundColor': colors['fullBG']})
 
 
@app.callback(
   Output('graph','figure'),
   Output('graph_rating','figure'),
   Output('my-output','options'),
   Input(component_id='my-input', component_property='value'),
   prevent_initial_call=True
)

def update(value):
  dfb=dfc[dfc["Country"]==value].values[0]
  v= dfb[0]
  dfd=df[df['Country Code']==v]
  dfcity=df_merge[df_merge["Country"]==value]
  figure= px.area(dfcity.groupby("City")['Aggregate rating'].mean(),y="Aggregate rating",title= 'Aggregate Ratings vs City')
  figure2=px.area(dfcity.groupby("City")['Votes'].sum(),y="Votes",title= 'No of Ratings vs City')
  return figure,figure2, [{'label':i,'value':i} for i in dfd['City'].unique()]


@app.callback(
   Output('graph3','figure'),
   
   Input(component_id='my-output', component_property='value'),
   prevent_initial_call=True
)
def upd(value):
  df_city=dfr[dfr['City']==value]
  fig= px.bar(df_city,x='Cuisines', y='Amount_In_Rupees',title= 'Rupee vs Cuisines', text_auto=True)
  return fig
if __name__ == '__main__':
   app.run_server(debug=True)

Dash app running on:


<IPython.core.display.Javascript object>